In [1]:
import nbimporter;
from UTILS.io_utils import df_from_csv, plot_save_fig, df_to_csv, save_splitterings_to_file, print_splittering_results
from UTILS.data_preprocessing_py import normalize_data, denormalize_data, re_normalize_data, time_resampling_old, fix_bad_samples,collect_segments_statistics, group_collect_statistics, copy_previous_for_nan
from UTILS.anns_utils_py import df_to_shifted_tables, build_multiparamiters_model, predict_time_window_with_knowns,  predictions_to_df
from UTILS.diagrams_utils import  plot_all_dfs
from UTILS.article_utils import for_article_description_df, correlation_coef 
from UTILS.data_reformat import group_data_by, split_data_by 
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
!pip install pmdarima
import pmdarima as pm

from keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
matplotlib.style.use(list(plt.style.available)[11])
%matplotlib inline
plt.close('all');

FILE_NAME = '6000_frames_20221124+25_new'
# FILE_NAME = '6000_frames_20221124'

COLUMN_NAMES_STATISTICS = ['Timestamp', 'Battery cell voltage', 'ActualSpeed_L', 'ActualSpeed_R',
                'Current segment', 'Speed','Cumulative energy consumption',
               'Momentary current consuption', 'Momentary energy consumption', 'Momentary power consumption'

###only with mass 
               ,'Weight statuses - front left strain gauge weight', 'Weight statuses - front right strain gauge weight',
               'Weight statuses - rear left strain gauge weight', 'Weight statuses - rear right strain gauge weight'
##
                          ]

STATIC_PARAMETERS =  ['Segment', 'Samples count', 'Duration', 'Voltage delta','Mass']

PREDICTED_PARAMETERS = ['Start segment voltage']

# A->D ->C ->B ->A
##-1##10.10.2022 12;100;120,8 C->B # 116 A->B 59;8 C->B 67,9; B->A 75,10-; A->D 84,10; D->C 93,7 
# -3 #11.10.2022 C->B 58,8; B->A 65,10; A->D 74,10 ; D->C 84 C->B 83,8 B->A 90,8; D->C 97,10; A->D 106,10; D->C 116

CUT = 0;
END_READING_STATISTIC_DATA =300#322
COUNT_NEURONS_HIDDEN_NEURON = 6
COUNT_NEURONS = 12 
LEARN_COUNT = 10#290#283, 150;
EPOCH_COUNT = 100# 120;
PREDICT_COUNT = 9 #30
COUNT_BATCH_SIZE = 11
ALL_DATA = 67000#52430#2500; 
BEGIN_READING_DATA =40000


In [2]:
# read data with file
df_origin = df_from_csv(FILE_NAME + '.pkl',path = 'data/Origin_pkl/')

# delete bed segments
df_origin = df_origin[df_origin.loc[:,'Current segment'] > 0]
df_orgin = df_origin.set_index(np.arange(df_origin.shape[0]))
# df1=df_origin.loc[0:ALL_DATA-1,COLUMN_NAMES_STATISTICS]
df1=df_origin.loc[BEGIN_READING_DATA:ALL_DATA-1,COLUMN_NAMES_STATISTICS]
df1 = df1.set_index(np.arange(df1.shape[0]))

### if we have column 'Mass' ####
n, m  = df1.shape

# Total weight of cargo
for i in range (n):
    df1.loc[i,'Mass'] = df1.loc[i,'Weight statuses - front left strain gauge weight'] + df1.loc[i,'Weight statuses - front right strain gauge weight'] + df1.loc[i,'Weight statuses - rear left strain gauge weight'] + df1.loc[i,'Weight statuses - rear right strain gauge weight']
df1 = df1.drop(columns=['Weight statuses - front left strain gauge weight', 'Weight statuses - front right strain gauge weight','Weight statuses - rear left strain gauge weight','Weight statuses - rear right strain gauge weight'])

# rename the columns of the segments to the same format
df1.rename(columns = {'Segment':'Current segment'}, inplace = True)

# df_origin.plot(y='Battery cell voltage', figsize=(10,6), title='Facebook Stock', ylabel='USD')
print(df1.shape)
df1.head()


(27000, 11)


,Timestamp,Battery cell voltage,ActualSpeed_L,ActualSpeed_R,Current segment,Speed,Cumulative energy consumption,Momentary current consuption,Momentary energy consumption,Momentary power consumption,Mass
0,2022-11-25 11:14:58.149311,42670.0,83.0,186.0,42.0,0.138347,887.0,7550.0,1285.0,322.0,200.0
1,2022-11-25 11:14:58.607783,42670.0,81.0,147.0,42.0,0.152438,888.0,7580.0,1302.0,323.0,200.0
2,2022-11-25 11:14:59.051414,42670.0,89.0,157.0,42.0,0.169731,888.0,7580.0,1302.0,323.0,200.0
3,2022-11-25 11:14:59.509562,42670.0,104.0,179.0,42.0,0.169731,888.0,7580.0,1302.0,323.0,200.0
4,2022-11-25 11:14:59.967345,42670.0,104.0,179.0,42.0,0.145392,888.0,7580.0,1302.0,323.0,200.0


In [3]:
# list segments
dfs_list = split_data_by(df1, column_name = 'Current segment', dfs = [],  label = "_m0kg"); ###пропускає перший сегмент???
df_stat = collect_segments_statistics(dfs_list, format = '%Y-%m-%d %H:%M:%S.%f')
# df_stat = df_stat.loc[0:LEARN_COUNT+PREDICT_COUNT+COUNT_NEURONS]
# print(dfs_list)
print(df_stat.shape)

status = copy_previous_for_nan(df1)


# list segments
dfs_list = split_data_by(df1, column_name = 'Current segment', dfs = [],  label = "_m0kg");
df_stat = collect_segments_statistics(dfs_list, format = '%Y-%m-%d %H:%M:%S.%f')
# print(df_stat.shape)

#  group data by segments
dfs_groups = group_data_by(df1, column_name = 'Current segment', dfs= {});#, label = "_m0kg");
# dfs_groups = group_data_by(df1, column_name = 'Segment', dfs= {});#, label = "_m0kg");
save_splitterings_to_file(dfs_groups, file_sufix="_test");
print_splittering_results(dfs_groups);

# df_statistics['Mass'] = 0########### only with mass

# averaged data for statistics
df_statistics = group_collect_statistics(dfs_groups, format = '%Y-%m-%d %H:%M:%S.%f')
df_statistics.index = df_statistics.Segment.values
# print(df_statistics.shape)
# df_statistics.head()

df_raw = df_stat[STATIC_PARAMETERS+PREDICTED_PARAMETERS]#+PREDICTED_PARAMETERS]
df_raw.dropna(axis=0)
df_raw = df_raw[df_raw.loc[:,'Segment'] > 0] ###without mass
# df_raw = df_raw[df_raw.loc[:,'Curent segment'] > 0] #With mass
# print(df_raw.shape)
df_raw.to_csv('Split_Segm.csv', sep = ';')
# df_raw.head(50)
df_raw_origin = df_raw

(703, 8)
seg42.0 22 [20, 55, 52, 124, 56, 53, 56, 49, 55, 49, 52, 57, 53, 535, 56, 63, 74, 56, 56, 53, 56, 48]
seg41.0 22 [43, 48, 47, 62, 49, 42, 49, 39, 49, 47, 50, 50, 48, 51, 64, 50, 403, 50, 42, 307, 49, 47]
seg9.0 22 [22, 23, 25, 21, 26, 26, 26, 24, 26, 26, 25, 25, 23, 26, 24, 22, 24, 26, 22, 26, 26, 26]
seg19.0 44 [10, 9, 9, 10, 10, 10, 7, 10, 9, 10, 10, 9, 10, 8, 9, 10, 10, 10, 9, 8, 10, 9, 10, 10, 9, 10, 9, 10, 8, 8, 10, 10, 9, 9, 9, 9, 10, 9, 9, 7, 9, 7, 10, 8]
seg56.0 45 [13, 126, 11, 14, 131, 13, 13, 128, 13, 14, 127, 13, 14, 128, 11, 12, 126, 9, 13, 123, 10, 11, 129, 14, 13, 126, 13, 12, 134, 13, 13, 130, 12, 11, 135, 125, 120, 121, 132, 131, 128, 107, 133, 121, 116]
seg51.0 44 [11, 8, 11, 11, 11, 11, 11, 11, 11, 9, 11, 9, 11, 10, 11, 10, 11, 11, 11, 9, 11, 8, 10, 11, 11, 11, 11, 10, 11, 11, 11, 10, 10, 11, 9, 12, 11, 10, 11, 11, 11, 11, 10, 10]
seg23.0 22 [20, 21, 22, 17, 19, 18, 22, 22, 21, 22, 22, 17, 22, 18, 21, 18, 22, 21, 21, 21, 22, 22]
seg59.0 22 [127, 134, 137, 13

In [4]:
df_statistics

,Segment,Duration,Duration Variance,Samples count,Voltage delta,Voltage delta variance,Mass
42.0,42.0,41.590909,41.590909,78.545455,-7.727273,16.495679,200.0
41.0,41.0,40.818182,40.818182,76.636364,-10.000000,9.534626,200.0
9.0,9.0,11.000000,11.000000,24.545455,-0.909091,12.026142,200.0
19.0,19.0,3.681818,3.681818,9.204545,-1.590909,6.375800,200.0
56.0,56.0,32.222222,32.222222,68.044444,-13.777778,18.533920,200.0
51.0,51.0,4.022727,4.022727,10.500000,-0.681818,6.874765,200.0
23.0,23.0,9.000000,9.000000,20.500000,-4.545455,8.381404,200.0
59.0,59.0,67.545455,67.545455,138.090909,-11.363636,16.038435,200.0
20.0,20.0,9.000000,9.000000,20.409091,3.636364,7.100227,200.0
48.0,48.0,4.045455,4.045455,10.590909,-0.227273,5.834317,200.0


In [5]:
# END_READING_STATISTIC_DATA =338
# COUNT_NEURONS_HIDDEN_NEURON = 11#4#4;
# COUNT_NEURONS = 12#12#12; 
# LEARN_COUNT = 300#283, 150;
# EPOCH_COUNT = 100#100; 120;
# PREDICT_COUNT =9 #30
# COUNT_BATCH_SIZE = 5

df_raw = df_raw_origin
START_READING_STATISTIC_DATA = END_READING_STATISTIC_DATA-LEARN_COUNT-COUNT_NEURONS-PREDICT_COUNT-1
df_raw = df_raw.loc[START_READING_STATISTIC_DATA:END_READING_STATISTIC_DATA,:]
df_raw = df_raw.set_index(np.arange(df_raw.shape[0]))
print(START_READING_STATISTIC_DATA,' ',END_READING_STATISTIC_DATA)


268   300


In [6]:
#---- for table A1 and table A2
# for_article_description_df(df1,df1.columns.values.tolist()[1:], table_name_params='origin')

In [7]:
#df1 userednene zminyty
# for_article_description_df(df1,df1.columns.values.tolist()[1:], table_name_params='without spontaneous outliers')

In [8]:
correlation_coef(df1, 'pearson', df1.columns.values.tolist()[1:])

pearson


/opt/homebrew/Caskroom/miniconda/base/envs/m2cnd-tf-pd2-p3/lib/python3.10/site-packages/pandas/io/formats/style.py:3819: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/opt/homebrew/Caskroom/miniconda/base/envs/m2cnd-tf-pd2-p3/lib/python3.10/site-packages/pandas/io/formats/style.py:3820: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,Battery cell voltage,ActualSpeed_L,ActualSpeed_R,Current segment,Speed,Cumulative energy consumption,Momentary current consuption,Momentary energy consumption,Momentary power consumption,Mass
Battery cell voltage,1.000000,0.000293,-0.001305,0.134546,-0.000175,-0.937212,0.151076,0.294022,0.284716,nan
ActualSpeed_L,0.000293,1.000000,0.967029,-0.046156,0.990278,0.000388,-0.002387,-0.003314,-0.002244,nan
ActualSpeed_R,-0.001305,0.967029,1.000000,-0.045356,0.988463,0.002082,-0.005844,-0.008078,-0.005872,nan
Current segment,0.134546,-0.046156,-0.045356,1.000000,-0.046270,-0.087338,0.162561,0.173765,0.175099,nan
Speed,-0.000175,0.990278,0.988463,-0.046270,1.000000,0.000837,-0.004472,-0.006147,-0.004359,nan
Cumulative energy consumption,-0.937212,0.000388,0.002082,-0.087338,0.000837,1.000000,-0.140159,-0.277943,-0.268776,nan
Momentary current consuption,0.151076,-0.002387,-0.005844,0.162561,-0.004472,-0.140159,1.000000,0.959679,0.990333,nan
Momentary energy consumption,0.294022,-0.003314,-0.008078,0.173765,-0.006147,-0.277943,0.959679,1.000000,0.971664,nan
Momentary power consumption,0.284716,-0.002244,-0.005872,0.175099,-0.004359,-0.268776,0.990333,0.971664,1.000000,nan
Mass,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [9]:
correlation_coef(df1, 'spearman', df1.columns.values.tolist()[1:])

spearman


/opt/homebrew/Caskroom/miniconda/base/envs/m2cnd-tf-pd2-p3/lib/python3.10/site-packages/pandas/io/formats/style.py:3819: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/opt/homebrew/Caskroom/miniconda/base/envs/m2cnd-tf-pd2-p3/lib/python3.10/site-packages/pandas/io/formats/style.py:3820: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,Battery cell voltage,ActualSpeed_L,ActualSpeed_R,Current segment,Speed,Cumulative energy consumption,Momentary current consuption,Momentary energy consumption,Momentary power consumption,Mass
Battery cell voltage,1.000000,-0.000250,-0.001659,0.067736,0.000166,-0.999793,-0.203192,0.349976,0.308263,nan
ActualSpeed_L,-0.000250,1.000000,0.939346,-0.082091,0.978314,0.000866,-0.012014,-0.020607,-0.015853,nan
ActualSpeed_R,-0.001659,0.939346,1.000000,-0.076506,0.967446,0.002202,-0.024629,-0.036284,-0.027657,nan
Current segment,0.067736,-0.082091,-0.076506,1.000000,-0.080358,-0.069738,0.145876,0.163698,0.166863,nan
Speed,0.000166,0.978314,0.967446,-0.080358,1.000000,0.000428,-0.014703,-0.024764,-0.016985,nan
Cumulative energy consumption,-0.999793,0.000866,0.002202,-0.069738,0.000428,1.000000,0.194249,-0.358142,-0.316740,nan
Momentary current consuption,-0.203192,-0.012014,-0.024629,0.145876,-0.014703,0.194249,1.000000,0.621743,0.814847,nan
Momentary energy consumption,0.349976,-0.020607,-0.036284,0.163698,-0.024764,-0.358142,0.621743,1.000000,0.802004,nan
Momentary power consumption,0.308263,-0.015853,-0.027657,0.166863,-0.016985,-0.316740,0.814847,0.802004,1.000000,nan
Mass,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [10]:
correlation_coef(df1, 'kendall', df1.columns.values.tolist()[1:])

kendall


,Battery cell voltage,ActualSpeed_L,ActualSpeed_R,Current segment,Speed,Cumulative energy consumption,Momentary current consuption,Momentary energy consumption,Momentary power consumption,Mass
Battery cell voltage,1.000000,-0.000483,-0.001248,0.046815,-0.000221,-0.993117,-0.149350,0.235832,0.209168,nan
ActualSpeed_L,-0.000483,1.000000,0.773193,-0.036829,0.874595,0.001011,-0.007244,-0.013768,-0.009619,nan
ActualSpeed_R,-0.001248,0.773193,1.000000,-0.035162,0.837174,0.001845,-0.015547,-0.023799,-0.017105,nan
Current segment,0.046815,-0.036829,-0.035162,1.000000,-0.036363,-0.048278,0.102309,0.113152,0.116841,nan
Speed,-0.000221,0.874595,0.837174,-0.036363,1.000000,0.000713,-0.007596,-0.014419,-0.008652,nan
Cumulative energy consumption,-0.993117,0.001011,0.001845,-0.048278,0.000713,1.000000,0.142973,-0.241320,-0.214784,nan
Momentary current consuption,-0.149350,-0.007244,-0.015547,0.102309,-0.007596,0.142973,1.000000,0.475866,0.665967,nan
Momentary energy consumption,0.235832,-0.013768,-0.023799,0.113152,-0.014419,-0.241320,0.475866,1.000000,0.659182,nan
Momentary power consumption,0.209168,-0.009619,-0.017105,0.116841,-0.008652,-0.214784,0.665967,0.659182,1.000000,nan
Mass,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.000000


In [11]:
# data normalization
df1, min_max = normalize_data(df_raw);
# df1 = df1.loc[0:LEARN_COUNT + PREDICT_COUNT +COUNT_NEURONS -1 ,:]
df1 = df1.loc[0:LEARN_COUNT + PREDICT_COUNT +COUNT_NEURONS -1 ,:]

####only with mass nan->0
n, m = df1.shape
for j in range(n):
    df1.iloc[j] = df1.iloc[j].fillna(0)

# print(df1.shape)
# df1.head()

In [12]:
# df1 = df1.apply (pd.to_numeric, errors='coerce')
# df1 = df1.dropna()
# df1 = df1.reset_index(drop=True)
# n, m = df1.shape
# for i in range(n):
#     for j in range(m): 
#         np = 
#         if pd.np.nan: print(df1.iloc[i,j])
#         #df1 = df1.drop(j, axis=1)
#          #   df1 = df1.drop(columns=[j])
 

In [13]:
# breakdown into training and test samples and construction of the ANN model
x_train, y_train, x_test, y_test = df_to_shifted_tables(df1, COUNT_NEURONS, LEARN_COUNT)


 -- Segment
 -- Samples count
 -- Duration
 -- Voltage delta
 -- Mass
 -- Start segment voltage


In [14]:
# model = build_multiparamiters_model(COLUMN_NAMES, PREDICTED_PARAMETERS, COUNT_NEURONS, hiden_layer_neurons = COUNT_NEURONS_HIDDEN_NEURON)
model = build_multiparamiters_model(STATIC_PARAMETERS+PREDICTED_PARAMETERS, PREDICTED_PARAMETERS, COUNT_NEURONS, hiden_layer_neurons = COUNT_NEURONS_HIDDEN_NEURON)

# coll model ANN with file 
# model = tf.keras.models.load_model('my_model_8.h5')

#the beginning of the training time of SNM
import time
start = time.time()

# fit model ANN 
epochs_hist = model.fit(x_train, y_train[PREDICTED_PARAMETERS], epochs = EPOCH_COUNT, batch_size = COUNT_BATCH_SIZE)

# The end of the ANN training time
end = time.time()
print(start,' ',end)
Training_Speed = round(((end-start)/60),2)
print(Training_Speed,' min')


Epoch 1/100
1/1 [==============================] - 0s 22ms/step - loss: 0.3478 - mean_absolute_error: 0.5704
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3410 - mean_absolute_error: 0.5655
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3353 - mean_absolute_error: 0.5614
Epoch 4/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3302 - mean_absolute_error: 0.5578
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3253 - mean_absolute_error: 0.5542
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3204 - mean_absolute_error: 0.5507
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 0.3165 - mean_absolute_error: 0.5479
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 0.3132 - mean_absolute_error: 0.5457
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 0.3099 - mean_absolute_error: 0.5435
Epoch 10/100
1/1 [=========

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Inp_Segment (InputLayer)    [(None, 12)]                 0         []                            
                                                                                                  
 Inp_Samples_count (InputLa  [(None, 12)]                 0         []                            
 yer)                                                                                             
                                                                                                  
 Inp_Duration (InputLayer)   [(None, 12)]                 0         []                            
                                                                                                  
 Inp_Voltage_delta (InputLa  [(None, 12)]                 0         []                        

In [16]:
# model.plot_diagnostics(figsize=(10,8))

In [17]:
# # Model loss progress during training the model
# plt.plot(epochs_hist.history['loss'], color='blue', label='Training loss')
# plt.plot(epochs_hist.history['mean_absolute_error'], color='red', label='Validation loss')

# plt.title('Model loss progress during training the model')
# plt.xlabel('Epochs')
# plt.ylabel('Training loss / Validation Loss')
# plt.legend(['Training loss', 'Validation Loss'])
# plot_save_fig( FILE_NAME+'_weigh_loss.png', path = 'OUTPUT/Plots/');


In [18]:
# assessment of the quality of the model on the test set
scores = model.evaluate(x_test, y_test[PREDICTED_PARAMETERS])

# # Predict
predictions = model.predict(x_test)

# We calculate the fit
predictions_train = model.predict(x_train)

# predict with time windoss method
df_knowns = y_test.loc[LEARN_COUNT:LEARN_COUNT+PREDICT_COUNT+COUNT_NEURONS, STATIC_PARAMETERS];

# df_knowns = y_test
ptw = predict_time_window_with_knowns(y_train, model, PREDICTED_PARAMETERS, df_knowns)



1/1 [==============================] - 0s 9ms/step
(10, 6)
 -- Segment
 -- Samples count
 -- Duration
 -- Voltage delta
 -- Mass
 -- Start segment voltage


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [11]:
# the result of the forecast with data normalization
df_predictions_train = predictions_to_df(predictions_train, PREDICTED_PARAMETERS);
df_predictions_train.index = y_train.index;

# print(df_predictions_train.shape)
# df_predictions_train.head() #добре


NameError: name 'predictions_train' is not defined

In [ ]:
df_predicted_test = predictions_to_df(predictions, PREDICTED_PARAMETERS);
df_predicted_test.index = y_test.index;

# print(df_predicted_test.shape)
# df_predicted_test.head()


In [ ]:
# #________
# from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(y_test,ptw)

# print(cm)

# #________
# from sklearn.metrics import confusion_matrix,classification_report
# print(classification_report(y_test,ptw)) 

# #________
# import seaborn as sn

# cm=tf.math.confusion_matrix(labels=y_test,predictions=ptw)
# plt.figure(figsize = (10,7))
# sn.heatmap(cm,annot=True,fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('Truth')


In [ ]:
plot_all_dfs([y_train, y_test, df_predictions_train, df_predicted_test, ptw], PREDICTED_PARAMETERS, 0, -1, legends = ['Mesuared data train', 'Mesuared data test', 'Predicted train', 'Predicted test', 'Predicted TW'])
plot_save_fig( FILE_NAME+'_weigh.png', path = 'OUTPUT/Plots/', fig_names = PREDICTED_PARAMETERS);

In [ ]:
# statistically averaged data for forecast comparison
route_segments = list(map(np.trunc, denormalize_data(y_test[["Segment"]], min_max).values.flat))
df_statistics.index = df_statistics.Segment.values 
print(route_segments)
print(df_statistics.index)

df_knowns_2 = df_statistics.loc[route_segments, STATIC_PARAMETERS]
df_knowns_2.index = df_knowns.index
# df_knowns_2

# renormalize data
df_knowns_3 = re_normalize_data(df_knowns_2, min_max)[0]
df_knowns_3.Mass = 0
df_knowns_3.index = df_knowns.index


ptw2 = predict_time_window_with_knowns(y_train, model, PREDICTED_PARAMETERS, df_knowns_3) #######-mass
# df_knowns_3.tail(1)
# denormalize data

# denormalize data
ptw_2 = denormalize_data(ptw2, min_max)
ptw_1 = denormalize_data(ptw, min_max)
y_test_1 = denormalize_data(y_test, min_max)
segments_labels = list(map(lambda x: '%03.0f' %(x), route_segments) )

# for statistics comparisions
route = route_segments
deltas = df_statistics.loc[route,'Voltage delta'].values
tmp = y_test_1['Start segment voltage'].values[0]
res = []
for x in deltas:
    tmp = tmp + x
    res.append(tmp)
#     print(' ',tmp,' ',x)
stat_voltage = pd.DataFrame(index = ptw_1.index)
stat_voltage.loc[:, ptw.columns[0]] = res
# stat_voltage

# # only predicted values with normalization
# plot_all_dfs([y_test, ptw, ptw2], PREDICTED_PARAMETERS, 0, -1, legends = ['Mesuared data test',  'Predicted TW1', 'Predicted TW2'])



In [ ]:
# only predicted values denormalization
plot_all_dfs([y_test_1, ptw_1, ptw_2, stat_voltage], PREDICTED_PARAMETERS, 0, -1, legends = ['Mesuared data test',  'Predicted TW1', 'Predicted TW2', 'Statistical voltage'])
    
segments_labels = list(map(lambda x: '%03.0f' %(x), route_segments) )
plt.xticks(ptw.index, segments_labels, rotation='vertical')
plot_save_fig( FILE_NAME+'_weigh1.png', path = 'OUTPUT/Plots/', fig_names = PREDICTED_PARAMETERS);

In [ ]:
# # errors of forecast results
# print('\n\nForecast Methods Comparision Report of the battery voltage fall.')

# route = route_segments

# route_info = pd.DataFrame()
# overall_duration = df_statistics.loc[route].Duration.sum()
# route_info.loc[0, 'Path through the %d segments' % (len(route))] = ' '.join(segments_labels); 
# route_info.loc[0, 'Overall duration (sec)'] = overall_duration;
# route_info.loc[0, 'Max overall duration'] = overall_duration + 3*df_statistics.loc[route, 'Voltage delta variance'].sum()
# route_info.loc[0, 'Battery voltage max fall (fact)'] = y_test_1.iloc[:, 0].min()- y_test_1.iloc[:, 0].max() 
# route_info.loc[0, 'Battery voltage max fall (ANN:TW1)'] = ptw_1.iloc[:, 0].min()- ptw_2.iloc[:, 0].max() 
# route_info.loc[0, 'Battery voltage max fall (ANN:TW2)'] = ptw_2.iloc[:, 0].min()- ptw_2.iloc[:, 0].max() 
# route_info.loc[0, 'Battery voltage max fall (stat)'] = df_statistics.loc[route,'Voltage delta'].sum()

# route_info.loc[0, 'Battery voltage max fall abs(fact)'] = abs(y_test_1.iloc[:, 0].min()- y_test_1.iloc[:, 0].max()) 
# route_info.loc[0, 'Battery voltage max fall abs(ANN:TW1)'] = abs(ptw_1.iloc[:, 0].min()- ptw_2.iloc[:, 0].max()) 
# route_info.loc[0, 'Battery voltage max fall abs(ANN:TW2)'] = abs(ptw_2.iloc[:, 0].min()- ptw_2.iloc[:, 0].max()) 
# route_info.loc[0, 'Battery voltage max fall abs(stat)'] = (abs(df_statistics.loc[route,'Voltage delta'])).sum()

# route_info.loc[0, 'Final battery voltage (fact)'] = y_test_1['Start segment voltage'].iloc[-1]
# route_info.loc[0, 'Final battery voltage (ANN:TW1)'] = ptw_1.iloc[-1, 0] 
# route_info.loc[0, 'Final battery voltage (ANN:TW2)'] = ptw_2.iloc[-1, 0] 
# route_info.loc[0, 'Final battery voltage (stat)'] = stat_voltage.iloc[-1, 0]
# route_info.loc[0, '------'] = 0

# route_info.loc[0, 'MSE (ANN:TW1)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS], squared=True),2)
# route_info.loc[0, 'MSE (ANN:TW2)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS], squared=True),2)
# route_info.loc[0, 'MSE (ANN:stat)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS], squared=True),2)

# route_info.loc[0, 'MAPE (ANN:TW1) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS])*100),2)
# route_info.loc[0, 'MAPE (ANN:TW2) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS])*100),2)
# route_info.loc[0, 'MAPE (ANN:stat) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS])*100),2)

# route_info.loc[0, 'MAE (ANN:TW1)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS]),2)
# route_info.loc[0, 'MAE (ANN:TW2)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS]),2)
# route_info.loc[0, 'MAE (ANN:stat)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS]),2)

# route_info.transpose()

In [ ]:
# Table6
print('Path through the %d segments' % (len(route)), ' '.join(segments_labels)); 
route = route_segments

route_info = pd.DataFrame()
overall_duration = df_statistics.loc[route].Duration.sum()
route_info.loc[0, 'Neurons in the Hidden Layers'] = COUNT_NEURONS_HIDDEN_NEURON 

route_info.loc[0, 'MSE (ANN:TW1)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS], squared=True),2)
route_info.loc[0, 'MSE (ANN:TW2)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS], squared=True),2)
route_info.loc[0, 'MSE (ANN:stat)'] = round(mean_squared_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS], squared=True),2)

route_info.loc[0, 'MAPE (ANN:TW1) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS])*100),2)
route_info.loc[0, 'MAPE (ANN:TW2) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS])*100),2)
route_info.loc[0, 'MAPE (ANN:stat) %'] = round((mean_absolute_percentage_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS])*100),2)

route_info.loc[0, 'MAE (ANN:TW1)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], ptw_1[PREDICTED_PARAMETERS]),2)
route_info.loc[0, 'MAE (ANN:TW2)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], ptw_2[PREDICTED_PARAMETERS]),2)
route_info.loc[0, 'MAE (ANN:stat)'] = round(mean_absolute_error(y_test_1[PREDICTED_PARAMETERS], stat_voltage[PREDICTED_PARAMETERS]),2)

route_info.loc[0, 'Training Speed min'] = Training_Speed
route_info.loc[0, 'Learn count'] = LEARN_COUNT 
route_info.loc[0, 'Epoch count'] = EPOCH_COUNT 
route_info.loc[0, 'Batch size'] = COUNT_BATCH_SIZE 


route_info

In [ ]:

# save model in file
model.save('my_model_С-B_11_11.h5') 
